In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import time
import scipy.optimize as opt
from selenium.webdriver.common.by import By
import numpy as np
import astropy.units as u
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
path = '/Users/wendy/_Lib/chromedriver_mac_arm64/chromedriver'

In [3]:
driver = webdriver.Chrome(path)


/var/folders/1c/5vhp7f916wv452z62sq58lzc0000gn/T/ipykernel_16504/2459958106.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [4]:
driver.get('https://smm.misis.ru/CPW-resonator-coupling/')


In [5]:
def set_epsilon(epsilon):
    epsi = driver.find_element(By.ID, 'epsilon')
    epsi.clear()
    epsi.send_keys('11.45')

In [6]:
def single_cpw_impedance(pin, gap):
    input_box = driver.find_element(By.ID, 'conductor-gap-widths-elements')
    AB = input_box.find_element(By.NAME, 'AB')
    BC = input_box.find_element(By.NAME, 'BC')
    CD = input_box.find_element(By.NAME, 'CD')

    AB.clear()
    AB.send_keys(gap)

    CD.clear()
    CD.send_keys(gap)

    BC.clear()
    BC.send_keys(pin)


    impedance_box = driver.find_element(By.ID, 'impedance-data')
    impedance = impedance_box.get_attribute('innerHTML')



    string = impedance
    match = re.search(r'Z0:(\d+\.\d+)', string)

    if match:
        number = float(match.group(1))
        # print(number)
    else:
        print("No match found.")


    return number

    

In [7]:
set_epsilon(11.45)

In [8]:
single_cpw_impedance(12.4, 6.99)

49.99140914897713

In [9]:
def find_optimal_gap(pin):
    def f(gap):
        time.sleep(0.01)
        return single_cpw_impedance(pin, gap)-50
    gap_val = opt.brentq(f, 0, 10)
    return round(gap_val, 2)

In [10]:
#for the transmission line, the optimal gap is:
find_optimal_gap(12.4)

6.99

In [ ]:
#for the resonator:
find_optimal_gap(12.4)

In [11]:
resonator_pin = 12.4
transmission_line_pin = 12.4
resonator_gap = find_optimal_gap(resonator_pin)
transmission_line_gap = find_optimal_gap(transmission_line_pin)

After getting optimal values for the individual transmissionline/resonators, we optimize the coupling between them.

In [12]:
def set_up_structure(coupling_space,resonator_pin = resonator_pin,resonator_gap=resonator_gap,transmission_line_pin=transmission_line_pin, transmission_line_gap=transmission_line_gap):
    button = driver.find_element(By.ID, 'add-conductor')
    button.click()
    button.click()
    time.sleep(0.5)
    conductor1 = driver.find_element(By.ID, 'tc3')
    conductor1.click()
    conductor1.click()
    time.sleep(0.5)

    input_box = driver.find_element(By.ID, 'conductor-gap-widths-elements')
    AB = input_box.find_element(By.NAME, 'AB')
    BC = input_box.find_element(By.NAME, 'BC')
    CD = input_box.find_element(By.NAME, 'CD')
    DE = input_box.find_element(By.NAME, 'DE')
    EF = input_box.find_element(By.NAME, 'EF')
    FG = input_box.find_element(By.NAME, 'FG')
    GH = input_box.find_element(By.NAME, 'GH')

    AB.clear()
    AB.send_keys(transmission_line_gap)

    CD.clear()
    CD.send_keys(transmission_line_gap)

    BC.clear()
    BC.send_keys(transmission_line_pin)
    time.sleep(0.5)

    
    DE.clear()
    DE.send_keys(coupling_space)

    EF.clear()
    EF.send_keys(resonator_gap)

    FG.clear()
    FG.send_keys(resonator_pin)

    GH.clear()
    GH.send_keys(resonator_gap)
    

In [13]:
def change_gap(coupling_space):
    input_box = driver.find_element(By.ID, 'conductor-gap-widths-elements')
    DE = input_box.find_element(By.NAME, 'DE')
    DE.clear()
    DE.send_keys(coupling_space)

In [14]:
def set_up_resonator():
    ztl1 = driver.find_element(By.ID, 'Zt1_short')
    ztl1.click()
    ztl2 = driver.find_element(By.ID, 'Zt2_open')
    ztl2.click()

    SC_coupler_len = driver.find_element(By.ID, 'lc')
    SC_coupler_len.clear()
    SC_coupler_len.send_keys('0.12')

    SC_coupler_l1 = driver.find_element(By.ID, 'l1')
    SC_coupler_l1.clear()
    SC_coupler_l1.send_keys('0')

    SC_coupler_l2 = driver.find_element(By.ID, 'l2')
    SC_coupler_l2.clear()
    SC_coupler_l2.send_keys('4')

In [15]:
set_up_structure(5)



In [16]:
set_up_resonator()

In [17]:
def find_resonator_length(freq):
    def f(length):
        time.sleep(0.01)
        SC_coupler_len = driver.find_element(By.ID, 'lc')
        SC_coupler_len.clear()
        SC_coupler_len.send_keys(length)
        return single_cpw_impedance(12.4, 9)-freq
    length_val = opt.brentq(f, 0, 10, xtol=0.001)
    return round(length_val, 2)

In [18]:
def find_resonator_specs():

    table = driver.find_element(By.ID, 'resonances')
    html = table.get_attribute('innerHTML')
    soup = BeautifulSoup(html, 'html.parser')

    # Extracting column names from HTML table headers
    cols = []
    for th in soup.find_all('th'):
        cols.append(th.text.strip())
        
    # Extracting table rows from HTML table body
    data = []
    for tr in soup.find_all('tr')[1:]:
        row = []
        for td in tr.find_all('td'):
            row.append(td.text.strip())
        data.append(row)

    # Creating pandas dataframe from table rows and columns
    df = pd.DataFrame(data, columns=cols)

    return df
    
    

In [19]:
freq = float(find_resonator_specs()['Frequency, GHz'][0])

In [20]:
def find_freq(len):
    SC_coupler_l2 = driver.find_element(By.ID, 'l2')
    SC_coupler_l2.clear()
    SC_coupler_l2.send_keys(len)

    return float(find_resonator_specs()['Frequency, GHz'][0])

In [21]:
def find_resonator_length(freq):
    def f(length):
        time.sleep(0.01)
        SC_coupler_len = driver.find_element(By.ID, 'l2')
        SC_coupler_len.clear()
        SC_coupler_len.send_keys(length)
        return float(find_resonator_specs()['Frequency, GHz'][0])-freq
    length_val = opt.brentq(f, 0, 10, xtol=0.001)
    return round(length_val, 2)

    

In [ ]:
l = find_resonator_length(6.4)

In [ ]:
freq_len = {}
for freqs in np.arange(6.4,8,0.2):
    freq_len[freqs] = find_resonator_length(freqs)

In [ ]:
freq_len

In [22]:
def find_Q(gap,freq):

    change_gap(gap)
    
    length = find_resonator_length(freq)
    SC_coupler_len = driver.find_element(By.ID, 'l2')
    SC_coupler_len.clear()
    SC_coupler_len.send_keys(length)

    
    return float(find_resonator_specs()['Quality factor'][0])

    


In [23]:
Q = find_Q(1,6.4)

In [24]:
kappa = 2*np.pi*freq*u.GHz/Q

In [25]:
kappa.to(u.MHz)

<Quantity 1.23629569 MHz>

In [26]:
def find_optimal_gap(freq):
    def kappa(gap):
        time.sleep(0.01)
        return (2*np.pi*freq*u.GHz/find_Q(gap,freq)-1*u.MHz).to(u.MHz).value
    gap_val = opt.brentq(kappa, 0, 10, xtol=0.001)
    return round(gap_val, 2)

In [27]:
find_optimal_gap(6.4)

1.25

In [28]:
gaps = {}

for freq in np.arange(6.4,8.4,0.2):
    gaps[freq] = find_optimal_gap(freq)

In [29]:
gaps

{6.4: 1.25,
 6.6000000000000005: 1.54,
 6.800000000000001: 1.86,
 7.000000000000001: 2.19,
 7.200000000000001: 2.54,
 7.400000000000001: 2.88,
 7.600000000000001: 3.25,
 7.800000000000002: 3.61,
 8.000000000000002: 3.99,
 8.200000000000003: 4.36}

In [30]:
dat = pd.DataFrame(gaps, index = [0])
dat.to_csv('gaps_um.csv')

In [34]:
dat.values[0]

array([1.25, 1.54, 1.86, 2.19, 2.54, 2.88, 3.25, 3.61, 3.99, 4.36])

In [35]:
data_all = pd.read_csv('/Users/wendy/Desktop/Wendy-qiskit-code/data/educated_guess_0403.csv')

In [ ]:
gaps_all = [1.25, 1.54, 1.86, 2.19, 2.54, 2.88, 3.25, 3.61]

In [36]:
data_all['Coupling_gap_feedline(um)'] = dat.values[0]

In [39]:
data_all.drop(columns = ['Unnamed: 0'], inplace = True)

In [45]:
data_all.to_csv(r'/Users/wendy/Desktop/Wendy-qiskit-code/data/educated_guess_all.csv')

In [ ]:
conductor1 = driver.find_element(By.ID, 'tc3')

In [ ]:
conductor1.click()

In [ ]:
conductor1.click()

In [ ]:
input_box = driver.find_element(By.ID, 'conductor-gap-widths-elements')
AB = input_box.find_element(By.NAME, 'AB')
BC = input_box.find_element(By.NAME, 'BC')
CD = input_box.find_element(By.NAME, 'CD')
DE = input_box.find_element(By.NAME, 'DE')
EF = input_box.find_element(By.NAME, 'EF')
FG = input_box.find_element(By.NAME, 'FG')
GH = input_box.find_element(By.NAME, 'GH')

In [ ]:
AB.clear()
AB.send_keys('9')
CD.clear()
CD.send_keys('9')

BC.clear()
BC.send_keys('15.95')

DE.clear()
DE.send_keys('5')

EF.clear()
EF.send_keys('7.4')

FG.clear()
FG.send_keys('12.5')

GH.clear()
GH.send_keys('7.4')



In [ ]:
epsi = driver.find_element(By.ID, 'epsilon')
epsi.clear()
epsi.send_keys('11.45')

In [ ]:
SC_coupler_len = driver.find_element(By.ID, 'lc')
SC_coupler_len.clear()
SC_coupler_len.send_keys('0.12')

SC_coupler_l1 = driver.find_element(By.ID, 'l1')
SC_coupler_l1.clear()
SC_coupler_l1.send_keys('0')

SC_coupler_l2 = driver.find_element(By.ID, 'l2')
SC_coupler_l2.clear()
SC_coupler_l2.send_keys('4')



In [ ]:
import pandas as pd
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

# Extracting column names from HTML table headers
cols = []
for th in soup.find_all('th'):
    cols.append(th.text.strip())
    
# Extracting table rows from HTML table body
data = []
for tr in soup.find_all('tr')[1:]:
    row = []
    for td in tr.find_all('td'):
        row.append(td.text.strip())
    data.append(row)

# Creating pandas dataframe from table rows and columns
df = pd.DataFrame(data, columns=cols)

print(df)